In [22]:
import os
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import gradio as gr
from torch.utils.data import Dataset

In [23]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

In [25]:
def load_data(folder):
    texts = []
    labels = []
    for filename in os.listdir(folder):
        if filename.endswith(".txt"):
            with open(os.path.join(folder, filename), 'r') as file:
                texts.append(file.read())
                labels.append(1 if 'pos' in filename else 0)  # 1 for positive, 0 for negative
    return texts, labels
    



Load data

In [ ]:
train_texts, train_labels = load_data('/home/brandon/IU International University/NLP: Project/aclImdb/train')
test_texts, test_labels = load_data('/home/brandon/IU International University/NLP: Project/aclImdb/test')

Split the training data for validation

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

Initialize the tokenizer

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Tokenize the data

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

Convert the data to PyTorch datasets

In [ ]:
train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

Initialize the model

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.w

Training settings

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=100,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

Create the Trainer and train

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

In [ ]:
trainer.train()

/home/brandon/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 50


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


COMET INFO: No Comet API Key was found, creating an OfflineExperiment. Set up your API Key to get the full Comet experience https://www.comet.com/docs/python-sdk/advanced/#python-configuration
COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: keras, tensorboard, tensorflow, torch. Metrics and hyperparameters can still be logged using Experiment.log_metrics() and Experiment.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Using '/home/brandon/IU International University/NLP: Project/.cometml-runs' path as offline directory. Pass 'offline_directory' parameter into constructor or set the 'COMET_OFFLINE_DIRECTORY' environment variable to manually choose where to store offline experiment archives.
COMET ERROR: Failed to extract scalar from SummaryWriter.add_hparams()


  0%|          | 0/50 [00:00<?, ?it/s]

OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.


...

OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{'loss': 0.6847, 'learning_rate': 2.0000000000000003e-06, 'epoch': 20.0}
{'loss': 0.7048, 'learning_rate': 3e-06, 'epoch': 30.0}
{'loss': 0.6891, 'learning_rate': 4.000000000000001e-06, 'epoch': 40.0}




Training completed. Do not forget to share your model on huggingface.co/models =)




{'loss': 0.6719, 'learning_rate': 5e-06, 'epoch': 50.0}
{'train_runtime': 417.5416, 'train_samples_per_second': 0.239, 'train_steps_per_second': 0.12, 'train_loss': 0.6886120796203613, 'epoch': 50.0}


TrainOutput(global_step=50, training_loss=0.6886120796203613, metrics={'train_runtime': 417.5416, 'train_samples_per_second': 0.239, 'train_steps_per_second': 0.12, 'train_loss': 0.6886120796203613, 'epoch': 50.0})

Evaluate the model

In [ ]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples = 2
  Batch size = 64


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.6953861713409424,
 'eval_runtime': 2.0121,
 'eval_samples_per_second': 0.994,
 'eval_steps_per_second': 0.497,
 'epoch': 50.0}

In [ ]:
def predict_review(text):
    # Tokenize and encode the input text
    encodings = tokenizer(text, truncation=True, padding=True, return_tensors='pt')
    
    # Get the model's output
    output = model(**encodings)
    
    # Convert output to probabilities
    probabilities = torch.nn.functional.softmax(output.logits, dim=-1)
    
    # Get the predicted class
    predicted_class = torch.argmax(probabilities)
    
    return 'positive' if predicted_class.item() == 1 else 'negative'

In [ ]:
iface = gr.Interface(fn=predict_review,
                     inputs=gr.Textbox(lines=2, placeholder="Enter comment here..."),
                     outputs="text",
                     allow_flagging='never')

In [ ]:
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Running on public URL: https://82914d2becc8c8781b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
